In [ ]:
import numpy as np

In [ ]:
def preprocess(fname):

    def id_generator():
        id = np.int64(0)
        while True:
            yield id
            id += 1

    id_gen = id_generator()

    diskmap = []
    with open(fname, "r") as f:
        for line in f.readlines():
            for ci  in range(0, len(line.strip())-1, 2):
                file_id = next(id_gen)
                diskmap.extend(int(line[ci])*[file_id])
                diskmap.extend(int(line[ci+1]) *[-1])
            file_id = next(id_gen)
            diskmap.extend(int(line.strip()[-1])*[file_id])
    return np.array(diskmap)

def part1(diskmap):
    # total number of file blocks
    num_blocks = np.sum(diskmap > 0)

    first_empty_idx = 0

    while True:
        # first empty block index:
        first_empty_idx = (diskmap < 0).nonzero()[0][0]
        # last nonempty block index
        last_full_idx = (diskmap >= 0).nonzero()[0][-1]
        if first_empty_idx > last_full_idx:
            break
        diskmap[first_empty_idx] = diskmap[last_full_idx]
        diskmap[last_full_idx] = -1

    checksum = np.int64(0)

    for ni in np.where(diskmap >= 0)[0]:
        checksum += ni*diskmap[ni]
    return checksum

diskmap = preprocess("day09_example.txt")
print(diskmap)
part1_example_sol = part1(diskmap)
print(f"Part 1 solution for example data: {(part1_example_sol)}")
assert (part1_example_sol) == 1928

In [ ]:
diskmap = preprocess("day09_input.txt")
part1_sol = part1(diskmap)
print(f"Part 1 solution: {(part1_sol)}")

In [ ]:
def part2(diskmap):
    max_file_id = np.max(diskmap)

    for file_id in np.arange(max_file_id + 1)[-1:0:-1]:
        # attempt moving file file_id
        
        start_points = np.diff(diskmap, prepend=-1, append=np.inf) != 0
        # start_points = np.append(start_points, True)
        len_blocks = np.diff(np.where(start_points)[0])

        start_locs = np.where(start_points)[0]
        # length of file file_id

        # type of block:
        block_type = diskmap[start_locs[:-1]]
        # print(np.where(block_type == file_id)[0])
        file_loc = start_locs[np.where(block_type == file_id)[0]][0]
        file_len = len_blocks[np.where(block_type == file_id)[0]][0]

        empty_locs = start_locs[np.where(block_type < 0)[0]]
        empty_lens = len_blocks[np.where(block_type < 0)[0]]
        
        for elen, eloc in zip(empty_lens, empty_locs):
            # print(elen,eloc, file_len)
            if elen >= file_len and eloc < file_loc:
                # print(f"Swapping file {file_id}, at {eloc}, len {elen}")
                diskmap[eloc:eloc + file_len] = file_id
                diskmap[file_loc:file_loc + file_len] = -1
                break


    checksum = np.int64(0)

    for ni in np.where(diskmap >= 0)[0]:
        checksum += ni*diskmap[ni]
    return checksum

diskmap = preprocess("day09_example.txt")
print(diskmap)
part2_example_sol = part2(diskmap)
print(f"Part 2 solution for example data: {(part2_example_sol)}")
assert (part2_example_sol) == 2858

In [ ]:
diskmap = preprocess("day09_input.txt")
part2_sol = part2(diskmap)
print(f"Part 2 solution for example data: {(part2_sol)}")